# 1. Loading the data

In [ ]:
# Loading a library for datasets
import pandas as pd

DATASET = 'WBI'
N_sets = 6
# DATASET = 'RIMPVC'
# N_sets = 5

def traces(i):
    """Get traces of dataset @i"""
    return pd.read_csv('data/WBI%s.csv' % i, index_col=0)

def derivs(i):
    '''Get derivatives of dataset @i'''
    return pd.read_csv('data/dWBI%s.csv' % i, index_col=0)

# Some code for later
from sklearn.preprocessing import MinMaxScaler 
minmax = MinMaxScaler()
from sklearn.preprocessing import StandardScaler
meanstd = StandardScaler()

In [ ]:
t0 = derivs(0)
print(t0.shape)
print(t0.head(10))

# 2. Data Visualization

## Traces on single dataset

In [ ]:
import pylab as plt
import numpy as np

#Define the function for plotting traces
def plot_all_traces(df, scaler=None):
    '''Plot all traces of dataset @df'''
    plt.figure(figsize=(20,30))
    if scaler is not None:
        df = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
    traces = df.values.copy()
    stretch = 2
    N = traces.shape[1]//2
    for i in range(traces.shape[1]):
        traces[:,i] += stretch * (i - N * (i > N-1))
    t1 = traces[:,:N]
    t2 = traces[:,N:]
    plt.subplot(121)
    plt.plot(t1, linewidth=1)
    plt.yticks(np.arange(0,N*stretch,stretch),labels=df.columns[:N])
    plt.subplot(122)
    plt.plot(t2, linewidth=1)
    plt.yticks(np.arange(0,N*stretch,stretch),labels=df.columns[N:])
    plt.show()
    plt.close()

In [ ]:
# Call the function for 1 dataset 
plot_all_traces(t0)

## Heatmap on single dataset

In [ ]:
import seaborn as sns

# Some colormaps
COLOR_MAPS = ['seismic', 'jet', 'Reds', 'flag']

#Define the function for plotting as a heatmap
def heatmap(df, scaler=None, cmap='flag'):
    '''
    Plot traces of df as a heatmap, 
    using @scaler for scaling 
    and @cmap for colors
    '''
    plt.figure(figsize=(20,30))
    if scaler is not None:
        df = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
    sns.heatmap(df.T, cmap=cmap, cbar_kws={"shrink": .5})
    plt.show()
    plt.close()

Run the code below and look at the result.  
The heatmap is obviously pretty unreadable, why ?  
Why not looking at this https://matplotlib.org/users/colormaps.html ?

In [ ]:
#Call the function for 1 dataset
heatmap(t0)

## Visualization of different scaling strategies

In [ ]:
NS = ['AVAL', 'AVBL']

def compare_norm(neurons=NS):
    """Compare the influence of different normalization methods on @neurons"""
    def plot_avab(df=t0, scaler=None):
        if scaler is not None:
            df = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
        for n in neurons:
            plt.plot(df[n], label=n)
        plt.legend()
        
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('Raw data')
    plot_avab()
    plt.subplot(132)
    plt.title('Min Max')
    plot_avab(scaler=minmax)
    plt.subplot(133)
    plt.title('Mean std')
    plot_avab(scaler=meanstd)
    plt.show()
    
compare_norm()
compare_norm(['AVAL', 'BAGL'])

## Visualization of all datasets

In [ ]:
all_names = set()
for i in range(N_sets):
    all_names = all_names.union(set(traces(i).columns))
LABELLED = [n  for n in all_names if any(c.isalpha() for c in n)]
LABELLED = sorted(LABELLED)
LABELLED.remove('bullshit')
LABELLED.remove('bullshit.1')

In [ ]:
def comp_amp(func, neur=LABELLED, stretch=1.7, scaler=None, title=''):
    '''
    Plot traces of neurons @neur 
    from data given by @func accross datasets
    @stretch defines the separation between traces
    @scaler defines the scaling for transforming the data
    @title will be the title of the figure
    '''
    t = 0
    N = len(neur)
    cmap = plt.cm.get_cmap('tab10')
    plt.figure(figsize=(45,30))
    for i in range(N_sets):
        df = func(i)
        if scaler is not None:
            df = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
        for j, n in enumerate(neur):
            try:
                plt.plot(t + df.index, df[n] + j*stretch, color=cmap(j%10))
            except:
                pass
        t += df.index[-1]
        plt.axvline(t, color='k', linestyle='--')
    
    plt.yticks(np.arange(0,N*stretch,stretch),labels=neur)   
    plt.tick_params(labeltop=False, labelright=True)
    plt.title(title, fontsize=20)
    plt.show()
            
comp_amp(traces)

## Influence of different scaling strategies over datasets

In [ ]:
def compare_norm(neurons):
    """Compare the influence of different normalization methods on @neurons accross datasets"""
    def plot_avab(df=t0, scaler=None):
        t = 0
        stretch = 2
        cmap = plt.cm.get_cmap('tab10')
        for i in range(N_sets):
            df = traces(i)
            if scaler is not None:
                df = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
            for j, n in enumerate(neurons):
                try:
                    plt.plot(t + df.index, df[n] + j*stretch, color=cmap(j%10))
                except:
                    pass
            t += df.index[-1]
            plt.axvline(t, color='k', linestyle='--')
        
    plt.figure(figsize=(20,10))
    plt.subplot(311)
    plt.title('Raw data')
    plot_avab()
    plt.subplot(312)
    plt.title('Min Max')
    plot_avab(scaler=minmax)
    plt.subplot(313)
    plt.title('Mean std')
    plot_avab(scaler=meanstd)
    plt.show()
    
compare_norm(['AVBL'])

# 3. Analysis

## 1. Correlation and distance

In [ ]:
N0 = 'AVAL'
N1 = 'AVAR'
N2 = 'DVA'

plt.figure(figsize=(10,10))
sns.scatterplot(t0[N0], t0[N1], label=N1)
sns.scatterplot(t0[N0], t0[N2], label=N2)
print('Correlation between %s and %s : %s' % (N0,N1,np.corrcoef(t0[N0], t0[N1])[0,1]))
print('Correlation between %s and %s : %s' % (N0,N2,np.corrcoef(t0[N0], t0[N2])[0,1]))
# plt.ylabel("$\frac{dF}{F_0}$")
plt.show()

Correlation & distance matrices

In [ ]:
from scipy.spatial.distance import pdist, squareform

def get_labelled(df):
    '''Return the name of labelled neurons in dataset @df'''
    return [n  for n in df.columns if any(c.isalpha() for c in n)]

def corr(df, vmin=None, vmax=None, cmap='flag'):
    '''Compute and plot correlation matrix on @df'''
    corr = df.corr()
    heat(corr, vmin, vmax, cmap)

def dist_comp(df):
    distances = pdist(df.T.values, metric='euclidean')
    return squareform(distances)
def dist(df, vmin=None, vmax=None, cmap='flag'):
    '''Compute and plot distance matrix on @df'''
    dist = pd.DataFrame(dist_comp(df), columns = df.columns, index=df.columns)
    heat(dist, vmin, vmax, cmap)
    
def heat(df, vmin=None, vmax=None, cmap='flag'):
    f, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(df, cmap=cmap, square=True, ax=ax, vmax=vmax, vmin=vmin, cbar_kws={"shrink": .5})
    

Run the code below and make it beautiful  
Do you see a difference between the 2 matrices ?  

In [ ]:
#Plot some matrices
names = get_labelled(t0)
corr(t0[names], cmap='flag')
plt.show()
dist(t0[names], cmap='flag')
plt.show()

## 2. PCA

In [ ]:
from sklearn.decomposition import PCA
from ipywidgets import interact
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
df = t0

@interact(scaler=[None, minmax, meanstd], cmap=COLOR_MAPS)
def pca(scaler=None, cmap='flag'):
    '''Perform pca on @df'''
    n = 10
    if scaler is not None:
        df2 = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
    else:
        df2 = df
    pca = PCA(n_components=n, whiten=True)
    pca.fit(df2)
    columns = ['pca_%i' % i for i in range(n)]
    df_pca = pd.DataFrame(pca.transform(df), columns=columns, index=df.index)
    %matplotlib notebook
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.scatterplot(x='pca_0', y='pca_1', data=df_pca, hue=df.index)
    ax = fig.gca(projection='3d')
    norm = matplotlib.colors.Normalize(0, df_pca.shape[0], clip=True)
    mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
    ax.scatter(df_pca['pca_0'], df_pca['pca_1'], df_pca['pca_2'], c=mapper.to_rgba(range(df_pca.shape[0])))
    plt.axis('off')
    plt.title('PCA transformation')
    plt.show()
    
    plt.figure(figsize=(10,5))
    sns.barplot(np.arange(n), pca.explained_variance_ratio_)
    plt.title('Amount of variance explained by components')
    plt.xlabel('# component')
    plt.ylabel('Proportion variance explained')
    plt.show()


## 3. Clustering

In [ ]:
%matplotlib inline
df = t0.corr()

@interact(method=['average', 'single', 'complete'], metric=['cityblock', 'euclidean'], ret=False)
def clustmap(method='average', metric='cityblock', ret=True):
    plt.figure(figsize = (15,15))
    cg = sns.clustermap(df, metric=metric, method=method, vmin=-1, vmax=1, cmap=sns.diverging_palette(220, 10, as_cmap=True))
    plt.show()
    if ret:
        return cg.dendrogram_col.reordered_ind

In [ ]:
order = df.columns[clustmap()]

Plot the traces and heatmap again using the clustering order

In [ ]:
### Your code here :